In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 30 18:10:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.4 MB/s eta 0:00:00


In [3]:
import os, torch
import pandas as pd
import time
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader, Dataset
from transformers import BertTokenizer, BertModel
import numpy as np
from torch import nn
from tqdm import tqdm
from torch.optim import SGD
from torch.nn.functional import softmax
from torch.utils.tensorboard import SummaryWriter

In [4]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [5]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/licenta/cls_output')

Mounted at /content/drive


In [6]:
print(os.listdir('./'))

['mono_en_de_dev_labels.npy', 'mono_en_de_dev_cls_output.npy', 'mono_en_de_test_cls_output.npy', 'mono_en_de_test_labels.npy', 'mono_en_es_dev_cls_output.npy', 'mono_en_es_dev_labels.npy', 'mono_en_es_test_labels.npy', 'mono_en_es_test_cls_output.npy', 'mono_en_de_train_labels.npy', 'mono_en_de_train_cls_output.npy', 'mono_en_es_train_labels.npy', 'mono_en_es_train_cls_output.npy']


In [7]:
def get_data_from_filename(filename):
  cls = np.load(filename + '_cls_output.npy')
  labels = np.load(filename + '_labels.npy')
  print(f" {filename} = {cls.shape} / {labels.shape}")
  return cls, labels

In [8]:
class DatasetBERT(Dataset):
    def __init__(self, X, Y, device):
        self.X = torch.tensor(X, device=device).float()
        self.Y = torch.tensor(Y, device=device).float()

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [22]:
    class MLP(nn.Module):
      def __init__(self):
        super().__init__()
        self.layer1 = nn.MaxPool1d(3, stride=2)
        self.layer2 = nn.Linear(383, 128)
        self.layer3 = nn.Linear(128, 2)
        self.activation = nn.RReLU()
        self.drop = nn.Dropout1d(p=0.15)
      def forward(self, x):
        out = self.activation(self.layer1(x))
        out = self.layer2(out)
        out = self.drop(out)
        out = self.layer3(out)
        return out

# class MLP(nn.Module):
#   def __init__(self):
#     super().__init__()
#     #model 1
#     self.layer1 = nn.Linear(768, 256)
#     self.layer2 = nn.Linear(256, 2)
#     # self.layer3 = nn.Linear(128, 2)
#     # self.activation = nn.ReLU()
#     self.activation = nn.Tanh()
#     self.drop = nn.Dropout1d(p=0.1)

#     #model 2
#     # self.layer1 = nn.MaxPool1d(3, stride=2)
#     # self.layer2 = nn.Linear(383, 128)
#     # self.layer3 = nn.Linear(128, 2)
#     # self.activation = nn.RReLU()
#     # self.drop = nn.Dropout1d(p=0.15)

#   def forward(self, x):
#     #recomand functia  CrossEntropyLoss deoarce avem clasificare cu doua clase(e binara chiar daca are 2)
#     #Astfel deoarece CrossEntropyLoss functioneaza cu logits nenormalizati vom returna outputul fara activare
#     # print(x.dst_type, self.layer1.type())
#     out = self.activation(self.layer1(x))
#     out = self.drop(out)
#     out = self.layer2(out)
#     # out = self.drop(out)
#     # out = self.layer3(out)
#     # out = self.drop(out)
#     # out = self.layer3(out)
#     return out

In [10]:
def get_dataloader_from_filename_batch(filename, batch_size):
  date_train_X, date_train_Y = get_data_from_filename(filename)

  dataset_train = DatasetBERT(date_train_X, date_train_Y, device)

  dataloader_train = DataLoader(dataset_train, batch_size, shuffle = True)
  return dataloader_train

In [11]:
BATCH_SIZE = 32

dataloader_train = get_dataloader_from_filename_batch('mono_en_de_train', BATCH_SIZE)
dataloader_validation = get_dataloader_from_filename_batch('mono_en_de_dev', BATCH_SIZE)

 mono_en_de_train = (29580, 768) / (29580,)
 mono_en_de_dev = (6336, 768) / (6336,)


In [12]:
dataloader_train

In [13]:
def _train_cnn_model_(cnn_model, optimizator_pentru_cnn, criterion_pentru_cnn, imgs_dataloader_train, epoca_de_rulare, grafic) :

    cnn_model.train()

    running_loss = 0.0

    for images_batch, labels_batch in imgs_dataloader_train:
        images_batch = images_batch.to(device)
        labels_batch = labels_batch.type(torch.LongTensor)
        labels_batch = labels_batch.to(device)

        output_labels = cnn_model(images_batch)
        # print(output_labels)
        # break

        loss_actual = criterion_pentru_cnn(output_labels, labels_batch)

        optimizator_pentru_cnn.zero_grad()

        loss_actual.backward()

        optimizator_pentru_cnn.step()

        running_loss += loss_actual.item() * len(images_batch)

    running_loss /= len(imgs_dataloader_train.dataset)
    # grafic.add_scalar(tag="Loss/Train", scalar_value=running_loss, global_step=epoca_de_rulare)
    # grafic.flush()


In [14]:
def _evaluate_cnn_model_(cnn_model, criterion_pentru_cnn, imgs_dataloader_validation, epoca_de_rulare, grafic):
    cnn_model.eval()

    running_loss = 0.0

    true_predictions_cnn = 0

    with torch.no_grad():

        for images_batch, labels_batch in imgs_dataloader_validation:
            images_batch = images_batch.to(device)
            labels_batch = labels_batch.type(torch.LongTensor)
            labels_batch = labels_batch.to(device)

            output_labels = cnn_model(images_batch)

            loss_actual = criterion_pentru_cnn(output_labels, labels_batch)

            scor = softmax(output_labels, dim=1)

            predicted_label = torch.argmax(scor, dim=1)

            true_predictions_cnn += (predicted_label == labels_batch).sum()

            running_loss += loss_actual.item() * len(images_batch)

        accuracy = true_predictions_cnn / len(imgs_dataloader_validation.dataset)
        running_loss = running_loss / len(imgs_dataloader_validation.dataset)

        print(f"acc is {accuracy} \t and loss is {running_loss}")
        grafic.add_scalar(scalar_value=accuracy, tag="Acc/Valid", global_step=epoca_de_rulare)
        grafic.add_scalar(scalar_value=running_loss, tag="Loss/Valid", global_step=epoca_de_rulare)

        grafic.flush()


In [16]:
def _run_cnn_model(_imgs_for_train_, _imgs_for_validation_, learning_rate_cnn, numar_epoci_model, nume_fisier_log):
    # best_loss = 99999.0
    # best_accuracy = 0.0

    cnn_model = MLP()

    cnn_model = cnn_model.to(device)

    optimizator_cnn = SGD(cnn_model.parameters(), lr = learning_rate_cnn)

    criterion_for_cnn = torch.nn.CrossEntropyLoss()

    _plotare_grafic_ = SummaryWriter(log_dir="../logs2/model_" + nume_fisier_log)

    for epoca_curenta in tqdm(range(numar_epoci_model), "Epochs") :
        _train_cnn_model_(cnn_model, optimizator_cnn, criterion_for_cnn, _imgs_for_train_, epoca_curenta, _plotare_grafic_)
        # break

        _evaluate_cnn_model_(cnn_model, criterion_for_cnn, _imgs_for_validation_, epoca_curenta, _plotare_grafic_)


In [ ]:
no_of_the_testing_model = 1
numar_epoci = 50
k = 6
for _learning_rate in [4*0.00001, 4*0.0001, 2*0.0001, 0.0001, 6*0.001, 4*0.001, 2*0.001, 0.001] :
  print('\n\n-----------------------------------\n\n')
  print(f'learning rate = {_learning_rate}')
  _run_cnn_model(dataloader_train, dataloader_validation, _learning_rate, numar_epoci,  str(k) + '_' + "test_cnn_" + str(no_of_the_testing_model))
  no_of_the_testing_model += 1




-----------------------------------


learning rate = 4e-05


Epochs:   2%|▏         | 1/50 [00:01<01:05,  1.33s/it]

acc is 0.5610795617103577 	 and loss is 0.6907469624220722


Epochs:   4%|▍         | 2/50 [00:02<01:03,  1.33s/it]

acc is 0.5552399158477783 	 and loss is 0.6905229190985361


...

acc is 0.6357322931289673 	 and loss is 0.6490413263590649


Epochs:  92%|█████████▏| 46/50 [01:01<00:05,  1.34s/it]

acc is 0.6377840638160706 	 and loss is 0.6485685826552035


Epochs:  94%|█████████▍| 47/50 [01:03<00:03,  1.33s/it]

acc is 0.6376262903213501 	 and loss is 0.6477333045367039
